# The Autonomous Vehicle Co-pilot
Here is a chatbot, empowered by GPT-3.5, running to help the vehicle driver. 

He is set as a co-pilot, analyzing the environment based on other multi-modality input, which has been processed as text so he can reason and identify potential risk. 


## Setup
Notice: if you tried to run this program on your linux server, 
and you are in China,
make sure that you opened the vpn. 
Otherwise the api account may be forbiddened!

In [1]:
import os
import openai
os.environ["http_proxy"]="127.0.0.1:10080"
os.environ["https_proxy"]="127.0.0.1:10080"

def read_from_file(filepath):
    with open(filepath, 'r') as file:
        return file.read().strip()

openai.api_key  = read_from_file('key/apikey.cn')

TEMPERATURE = 0

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo-16k-0613"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= TEMPERATURE, # this is the degree of randomness of the model's output
    )
    return response
    # return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo-16k-0613",temp = TEMPERATURE):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= temp, # this is the degree of randomness of the model's output
    )
    # print(str(response.choices[0].message["content"]))
    return response.choices[0].message["content"]

## test on the api

In [3]:
blind_guide = """ 
We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.
Next, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.
We are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection has only one lane, clearly marked. In front of us, there is a blue sedan driving at a moderate speed."""
chat_history = [
    { 'role':'system','content': blind_guide},
    {'role':'system', 'content':'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'},
    {'role':'user', 'content':'What is our current situation on the road?'},
]

# test_response = get_completion_from_messages(chat_history);print(test_response)

In [4]:
len(blind_guide)

792

In [5]:
sys_prompt_mot = read_from_file("key/mot_sys.md")
usr_prompt_mot = read_from_file("key/mot_usr.md")
START_USR = "Based on the following sequence of frames, provide a concise description of the road scene, capturing significant events or movements of people:"


# Data Integration
 Firstly, you'll need a way to convert the input from the vehicle's sensors (like LIDAR, RADAR, cameras, etc.) into a format that the chatbot can understand. This might involve using object detection algorithms to identify objects and their positions, then converting this data into a text description of the scene.


## buggy chatbot here
I need to find a good programming way of coding here

In [6]:
# def chat_with_assistant(user_input, context):
#     # Append the user input to the context
#     context.append({'role':'user', 'content':f"{user_input}"})
    
#     # Get the assistant's response
#     response = get_completion_from_messages(context)
    
#     # Append the assistant's response to the context
#     context.append({'role':'assistant', 'content':f"{response}"})
    
#     return context


# # Chat with the assistant
# context = chat_with_assistant('Hello, assistant!', chat_history)
# print(context)

# # Continue the conversation
# context = chat_with_assistant('Tell me a joke', context)
# print(context)


ChatGPT, based on the incoming road scenario data, I need you to assess the situation and provide us with critical information. 
## input
The road scenario data will be provided in {'context'}, it contains the continuous frame information of video from dashcam in the vehicle. The autonomous system provides you with information on road layout, people's locations, their distances and angles from the dashcam, the surfaces they are on, and the confidence level of each detection.
## output 
Your analysis should provide:
- an understanding of the context complexity (low, medium, or high), 
- the number of persons, cars, and bikes present in the scene, 
- an array detailing the danger level for each person, if applicable.
The output should be markdown format

This analysis should be grounded in the current context and make predictions for the short term future to assist in autonomous driving. The key goal here is to identify potential risks to enhance the safety and efficiency of our journey.

"You are a co-pilot AI designed to analyze road scenario data and provide critical information about the situation. Based on the video frames from the dashcam and the data provided by the autonomous system, you will assess the context complexity, count the number of persons, cars, and bikes in the scene, and evaluate the danger level for each person. Your goal is to identify potential risks to enhance the safety and efficiency of the journey. The output should be in markdown format.

In [7]:
sys_prompt = read_from_file("key/sys_prompt.md")
usr_prompt = read_from_file("key/usr_prompt.md")

def get_sys_msg(content:str): 
    system_msg = {"role": "system", "content": content}
    return  system_msg
def get_usr_msg(content:str): 
    msg = {"role": "user", "content": content}
    return msg


In [8]:
data = """INFO of 0016:
road 0 is at middle_down
sidewalk 1 is at right_down
sidewalk 2 is at right_down
person 0 is at middle_down
The [distance,angle] from person 0 to our dashcam is: [very close,89.97960036997033]
person 1 is at left_down
The [distance,angle] from person 1 to our dashcam is: [very close,-89.98811258767391]
person 2 is at left_down
The [distance,angle] from person 2 to our dashcam is: [very close,-89.98749824583163]
person 3 is at middle_down
The [distance,angle] from person 3 to our dashcam is: [very close,89.96852785395733]
person 4 is at left_down
The [distance,angle] from person 4 to our dashcam is: [very close,-89.99328360812777]
person 5 is at left_down
The [distance,angle] from person 5 to our dashcam is: [very close,-89.99173842984247]
person 6 is at left_down
The [distance,angle] from person 6 to our dashcam is: [very close,-89.99199265188705]
person 7 is at left_down
The [distance,angle] from person 7 to our dashcam is: [very close,-89.99241327380344]
Person 0 is on the road 0, sidewalk 1, sidewalk 2,his/her bbox is [1151.6272   683.91156 1259.8259  1029.4976 ]
Person 1 is on the road 0, sidewalk 1, sidewalk 2,his/her bbox is [520.33466 722.2189  556.2974  830.36035]
Person 2 is on the road 0, sidewalk 1, sidewalk 2,his/her bbox is [544.58014 737.0274  573.4954  827.68024]
Person 3 is on the road 0,his/her bbox is [1110.2614  723.2576 1128.289   741.3161]
Person 4 is on the road 0,his/her bbox is [205.18512 686.6346  222.12915 713.3797 ]
Person 5 is on the road 0, sidewalk 1,his/her bbox is [343.09924 702.2559  363.39508 733.56   ]
Person 6 is on the road 0,his/her bbox is [324.6748  691.9793  343.29242 718.48505]
Person 7 is on the road 0,his/her bbox is [291.81046 690.33636 306.7418  715.5451 ]
number of Surface mask, Road&sidewalk, People 's mask, actural people: (9, 3, 9, 8)"""

In [9]:
messages = [
    get_sys_msg(sys_prompt),
    get_usr_msg(usr_prompt),
    {"role": "user", "content": f"here's the current road scenario data: {data}"},
]
# response = get_completion_from_messages(messages)

Great, now I have a working chatGPT api as below:{{{
messages = [
    get_sys_msg(sys_prompt),
    get_usr_msg(usr_prompt),
    {"role": "user", "content": f"here's the current road scenario data: {frame_data}"},
]
response = get_completion_from_messages(messages)
}}}
For raw data stored in folder raw_data/ , there are info of video as below:
- raw_data/Info_Video_0001.txt
- raw_data/Info_Video_0002.txt
...
### task: I want you provide a double for-loop for me, here is the pesudo code:

for all the Info txt in raw_data folder{
    for each frame in the txt{
        read the frame info into variable 'frame_data'
        response = get_completion_from_messages
        append the response into 'LLM_data/Result_Video_xxxx', here the xxxx is current video id
    }
}


In [10]:
def read_frames_from_raw_file(file_path):
    frames = []
    current_frame_data = ''
    reading_frame = False

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('INFO of'):
                if reading_frame:
                    frames.append(current_frame_data.strip())
                    current_frame_data = ''
                reading_frame = True

            if reading_frame:
                current_frame_data += line

    # Adding the last frame if any
    if current_frame_data:
        frames.append(current_frame_data.strip())

    return frames



In [11]:
# import os
# import re
# from pathlib import Path

# raw_data_folder = "raw_data/"

# output_folder = "LLM_data/"

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# for filename in os.listdir(raw_data_folder):
#     if filename.startswith("Info_Video_") and filename.endswith(".txt"):
#         # the id may be 'bd99', '0001', 'eb61'
#         video_name = filename[-8:-4]
#         output_path = Path(f"LLM_data/Result_Video_{video_name}.txt")
        
#         frames = read_frames_from_raw_file(os.path.join(raw_data_folder, filename))
#         start_frame = 0
#         if output_path.exists():
#             print(f"Already handled {output_path}")
#             # Read the existing frames that have been processed
#             # with open(output_path, 'r') as result_file:
#             #     processed_frames = re.findall(r'### Frame (\d{4}) Analysis', result_file.read())
#             #     start_frame = len(processed_frames) # Start from the next unprocessed frame
#             #     frames = frames[start_frame:] # Excl
#             #     if frames is None:
#             #         continue
#             continue
#         with open(os.path.join(raw_data_folder, filename), 'r') as raw_file:
#             with open(os.path.join(output_folder, f"Result_Video_{video_name}.txt"), 'a') as result_file:
#                 print(f"Analyzing video {video_name} starting from frame {start_frame}")

#                 for frame_data in frames:
#                     messages = [
#                         get_sys_msg(sys_prompt), 
#                         get_usr_msg(usr_prompt), 
#                         {"role": "user", "content": f"here's the current road scenario data: {frame_data}"},
#                     ]
#                     response = get_completion_from_messages(messages)
                    
#                     result_file.write(response + '\n')
                    
#         print(f"Analysis completed for video {video_name}")

# print("All videos processed!")


In [12]:
import re

input_file = "./prep_data/0829_J2T/video_0194.txt"

def downsample_read_frame(file_path, step=2):
    """
    Temporally downsample the data based on frame number.

    Args:
    - file_path (str): Path to the file containing the data.
    - step (int): The step size for downsampling. Default is 2 (every second frame).

    Returns:
    - list of str: The downsampled data, each item is a string containing all the information for one frame.
    """
    
    with open(file_path, 'r') as f:
        lines = f.readlines()

    downsampled_data = []
    current_frame_block = []
    current_frame_number = None
    
    for line in lines:
        if line.startswith("### frame"):
            if current_frame_block:  # Save the previous frame block if there is one
                if current_frame_number % step == 0:
                    downsampled_data.append("".join(current_frame_block))
                current_frame_block = []
                
            current_frame_number = int(re.search(r"### frame (\d+)", line).group(1))
        current_frame_block.append(line)
        
    if current_frame_block and current_frame_number % step == 0:  # Don't forget the last block
        downsampled_data.append("".join(current_frame_block))

    return downsampled_data

# Test the function 
downsampled_frames = downsample_read_frame(input_file)
print(downsampled_frames)


['### frame 0000\npeople detected: 1,2,3,4,5\n1 is at [-21.9, 662.1, 90.5, 971.6] \n2 is at [1098.7, 677.2, 1136.0, 763.4] \n3 is at [848.2, 709.2, 877.5, 773.1] \n4 is at [180.3, 675.0, 212.7, 760.3] \n5 is at [1078.2, 684.8, 1108.4, 754.9] \nOn the right of our car, 2,5\nOn the left of our car,1,2,3,4,5\nhigh confidence: 1,2\nlow confidence: none \n\n', '### frame 0002\npeople detected: 2,3,4,5,7\n2 is at [1106.8, 664.7, 1143.3, 749.0] \n3 is at [850.1, 701.2, 879.6, 765.6] \n4 is at [166.5, 667.8, 193.9, 740.0] \n5 is at [1085.9, 675.7, 1112.3, 736.7] \n7 is at [646.1, 701.6, 664.8, 750.5] \nOn the right of our car, 2,5\nOn the left of our car,2,3,4,5,7\nhigh confidence: none\nlow confidence: none \n\n', '### frame 0004\npeople detected: 2,3,4,5,7\n2 is at [1115.0, 649.4, 1153.7, 738.7] \n3 is at [852.7, 687.0, 882.8, 752.9] \n4 is at [147.0, 655.9, 178.0, 737.8] \n5 is at [1093.1, 661.4, 1122.2, 728.9] \n7 is at [645.5, 689.4, 664.0, 737.8] \nOn the right of our car, 2,5\nOn the le

In [13]:
def read_frames_into_batch(file_path, step = 1, batch_size = 7):
    frames = []
    current_frame_data = ''
    reading_frame = False
    current_frame_number = 0

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('### frame'):
                if reading_frame:
                    if current_frame_number % step == 0:
                        frames.append(current_frame_data.strip())
                    current_frame_data = ''
                    
                reading_frame = True
                current_frame_number = int(line.split(' ')[-1])  # Assumes the frame number is the last item in the line

            if reading_frame:
                current_frame_data += line

    # Adding the last frame if any
    if current_frame_data and current_frame_number % step == 0:
        frames.append(current_frame_data.strip())
  # Batch frames
    batches = [frames[i:i + batch_size] for i in range(0, len(frames), batch_size)]
    concatenated_batches = ['\n'.join(batch) for batch in batches]
    return concatenated_batches



# ChatBOT 2.0 + MOT

In [14]:
sys_prompt_mot = read_from_file("key/mot_sys.md")
usr_prompt_mot = read_from_file("key/mot_usr.md")
START_USR = "Based on the following sequence of frames, provide a description of the road scene, capturing significant events or movements of people:\n"


In [17]:
from pathlib import Path
import os

def analyze_videos_with_llm(raw_data_folder="./prep_data/0830_Select/", output_folder="LLM_data/0830_MOT/", 
                            sys_prompt="System message", usr_prompt="User prompt"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(raw_data_folder):
        if filename.endswith(".txt"):
            video_name = os.path.basename(filename).split('.')[0]
            input_path = Path(f"{raw_data_folder}/{filename}")
            output_path = Path(f"{output_folder}/llm_{video_name}.txt")
            
            if output_path.exists():
                print(f"Already handled {output_path}")
               
                # Count existing batches to skip them
                with open(output_path, 'r') as f:
                    content = f.read()
                    if '<-end_processing->' in content:
                        print(f"Finished handling {output_path}")
                        continue
                    else:
                        existing_batches = content.count('<-batch_')
                
                if existing_batches > 0:
                    print(f"Resuming from batch {existing_batches}")
                else:
                    existing_batches = 0  # Reset to 0 if no existing batch is found
            else:
                existing_batches = 0
            
            batches = read_frames_into_batch(input_path,batch_size=10)

            # Open file in append mode ('a') to continue writing if the file exists
            with open(output_path, 'a') as result_file:
                print(f"Analyzing video {video_name} starting from batch {existing_batches}")
                for i, batch in enumerate(batches[existing_batches:], start=existing_batches):
                    print(f"batch:{i},  {batches[i][:14]}")
                    messages = [
                        get_sys_msg(sys_prompt),
                        get_usr_msg(usr_prompt),
                        {"role": "user", "content": f"{START_USR} {batch}"},
                    ]
                    response = get_completion_from_messages(messages)
                    
                    result_file.write(f'<-batch_{i}->\n' + response + '\n' )
                #end
                result_file.write(f'\n<-end_processing->\n' )  
                
            print(f"Analysis completed for video {video_name}")

    print("All videos processed!")

# Usage example:
analyze_videos_with_llm(sys_prompt = sys_prompt_mot, usr_prompt = usr_prompt_mot)


Already handled LLM_data\0830_MOT\llm_b1c9c847-3bda4659.txt
Finished handling LLM_data\0830_MOT\llm_b1c9c847-3bda4659.txt
Already handled LLM_data\0830_MOT\llm_b1d10d08-c35503b8.txt
Finished handling LLM_data\0830_MOT\llm_b1d10d08-c35503b8.txt
Already handled LLM_data\0830_MOT\llm_b1d22ed6-f1cac061.txt
Finished handling LLM_data\0830_MOT\llm_b1d22ed6-f1cac061.txt
Already handled LLM_data\0830_MOT\llm_b1dac7f7-6b2e0382.txt
Finished handling LLM_data\0830_MOT\llm_b1dac7f7-6b2e0382.txt
Already handled LLM_data\0830_MOT\llm_b1f4491b-33824f31.txt
Finished handling LLM_data\0830_MOT\llm_b1f4491b-33824f31.txt
Already handled LLM_data\0830_MOT\llm_b1f4491b-9958bd99.txt
Finished handling LLM_data\0830_MOT\llm_b1f4491b-9958bd99.txt
Already handled LLM_data\0830_MOT\llm_b2ae0446-4b0bfda7.txt
Finished handling LLM_data\0830_MOT\llm_b2ae0446-4b0bfda7.txt
Already handled LLM_data\0830_MOT\llm_b2be7200-b6f7fe0a.txt
Finished handling LLM_data\0830_MOT\llm_b2be7200-b6f7fe0a.txt
Already handled LLM_data

problem batch:

 b1d10d08-c35503b8, b1d22ed6-f1cac061, 
 Analyzing video b1f4491b-33824f31 starting from batch 42
### frame 0294
peopl
Analyzing video b1f4491b-33824f31 starting from batch 43
### frame 0301


Analyzing video b1f4491b-9958bd99 starting from batch 44
### frame 0308

Analyzing video b2d22b2f-8302eb61 starting from batch 23
### frame 0161

Analyzing video b1c9c847-3bda4659 starting from batch 150
### frame 1050

Analyzing video b4542860-0b880bb4 starting from batch 40
### frame 028
InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4390 tokens. Please reduce the length of the messages.

Analyzing video cd26264b-22001629 starting from batch 16
### frame 0112
peopl

In [ ]:
def extract_information(input_text):
    # Find the position of "Therefore" in the text
    index = input_text.find("Therefore")
    
    # If "Therefore" is found, extract the text after it
    if index != -1:
        information_after_therefore = input_text[index:]
    else:
        information_after_therefore = "Keyword 'Therefore' not found in the text."

    return information_after_therefore

def write_to_file(output_text, file_name='output.txt'):
    with open(file_name, 'w') as file:
        file.write(output_text)

if __name__ == "__main__":
    # Read input text from a file
    with open('input.txt', 'r') as file:
        input_text = file.read()

    # Extract the information after "Therefore"
    output_text = extract_information(input_text)

    # Write the output to another text file
    write_to_file(output_text)

    print(f"Information after 'Therefore' has been written to {output_file_name}")


2. **Contextual Understanding:** Your chatbot should be capable of maintaining and updating a model of the car's surroundings based on this information. For example, it should keep track of the positions of other vehicles, pedestrians, and road features, updating these as new data comes in.

3. **Real-Time Interaction:** The chatbot should be capable of generating responses in real-time. Delays could be dangerous in a driving context.

4. **Proactive Alerting:** Your co-pilot chatbot should not just react to the driver's queries but should also proactively provide alerts and advice. For example, if it detects a pedestrian stepping onto the road, it should immediately alert the driver.

5. **Simulated Training:** Consider using simulated environments to train and test your chatbot before deploying it in a real vehicle. This can help you ensure that the chatbot behaves correctly in a wide range of situations.

6. **Safety Precautions:** Since the bot is serving in a co-pilot role, it should prioritize safety and assume the driver may be unaware of potential dangers. It should also be prepared to handle ambiguous situations by advising caution.

7. **User Customization:** Different drivers may have different preferences for how much information they want to receive, what kind of language the co-pilot uses, etc. Consider allowing users to customize the chatbot's behavior to some extent.

8. **Integration with Vehicle's Systems:** If possible, the chatbot could also be integrated with the vehicle's systems to provide even more functionality. For example, it could adjust the vehicle's speed or even take control in an emergency. However, this would require a high level of reliability and many legal and ethical considerations.

maybe this will help 

In [ ]:
video_info_str = """
Video file: video_0001.mp4
Resolution: 1920x1080
Number of frames: 600
Frames per second: 29.97002997002997
Duration (seconds): 20.02

Video file: video_0310.mp4
Resolution: 1920x1080
Number of frames: 150
Frames per second: 30.0
Duration (seconds): 5.0

Video file: video_0194.mp4
Resolution: 1920x1080
Number of frames: 540
Frames per second: 30.0
Duration (seconds): 18.0

Video file: video_0343.mp4
Resolution: 1920x1080
Number of frames: 720
Frames per second: 30.0
Duration (seconds): 24.0

Video file: video_0003.mp4
Resolution: 1920x1080
Number of frames: 210
Frames per second: 29.97002997002997
Duration (seconds): 7.007000000000001

Video file: video_0333.mp4
Resolution: 1920x1080
Number of frames: 210
Frames per second: 30.0
Duration (seconds): 7.0

Video file: video_0313.mp4
Resolution: 1920x1080
Number of frames: 600
Frames per second: 30.0
Duration (seconds): 20.0

Video file: video_0055.mp4
Resolution: 1920x1080
Number of frames: 210
Frames per second: 29.97002997002997
Duration (seconds): 7.007000000000001

Video file: video_0056.mp4
Resolution: 1920x1080
Number of frames: 270
Frames per second: 29.97002997002997
Duration (seconds): 9.009

Video file: video_0057.mp4
Resolution: 1920x1080
Number of frames: 240
Frames per second: 29.97002997002997
Duration (seconds): 8.008000000000001


"""

def parse_video_info(video_info_str):
    # Split the string into individual video blocks
    video_blocks = video_info_str.strip().split("\n\n")
    
    video_info_dict = {}
    
    for block in video_blocks:
        lines = block.split("\n")
        
        # Extract video file name
        video_file = lines[0].split(": ")[1]
        
        # Extract resolution
        resolution = lines[1].split(": ")[1]
        
        # Extract number of frames
        num_frames = int(lines[2].split(": ")[1])
        
        # Extract frames per second
        fps = float(lines[3].split(": ")[1])
        
        # Extract duration
        duration = float(lines[4].split(": ")[1])
        
        # Populate the dictionary
        video_info_dict[video_file] = {
            "resolution": resolution,
            "num_frames": num_frames,
            "fps": fps,
            "duration": duration
        }
    
    return video_info_dict

video_data = parse_video_info(video_info_str)
print(video_data)
